In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
mpsolution = mp.solutions.holistic
drawing = mp.solutions.drawing_utils

In [3]:
def Superimpose(image, landmarks):
    drawing.draw_landmarks(image, landmarks.face_landmarks, mpsolution.FACEMESH_CONTOURS, drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    drawing.draw_landmarks(image, landmarks.pose_landmarks, mpsolution.POSE_CONNECTIONS)
    drawing.draw_landmarks(image, landmarks.left_hand_landmarks, mpsolution.HAND_CONNECTIONS, drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    drawing.draw_landmarks(image, landmarks.right_hand_landmarks, mpsolution.HAND_CONNECTIONS, drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))

In [4]:
def BrightnessContrast(img, brightness = 0, contrast = 0):
     
    # getTrackbarPos returns the current
    # position of the specified trackbar.
    # brightness = cv2.getTrackbarPos('Brightness', 'test')
      
    # contrast = cv2.getTrackbarPos('Contrast', 'test')
  
    effect = controller(img, brightness, contrast)
  
    # The function imshow displays an image
    # in the specified window
    return effect

def controller(img, brightness=255, contrast=127):
    brightness = int((brightness - 0) * (255 - (-255)) / (510 - 0) + (-255))
    contrast = int((contrast - 0) * (127 - (-127)) / (254 - 0) + (-127))
  
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            max = 255
        else:
            shadow = 0
            max = 255 + brightness

        al_pha = (max - shadow) / 255
        ga_mma = shadow
  
        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(img, al_pha, img, 0, ga_mma)
    else:
        cal = img
  
    if contrast != 0:
        Alpha = float(131 * (contrast + 127)) / (127 * (131 - contrast))
        Gamma = 127 * (1 - Alpha)  
        # The function addWeighted calculates
        # the weighted sum of two arrays
        cal = cv2.addWeighted(cal, Alpha, cal, 0, Gamma)
  
    # putText renders the specified text string in the image.
    #cv2.putText(cal, 'B:{},C:{}'.format(brightness, contrast), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
  
    return cal

In [5]:
def detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return result

In [6]:
def Process(result):
    processed = dict()

    if result.pose_landmarks:
        temp = list(result.pose_landmarks.landmark)[0]
        landmarks = np.array([[(res.x - temp.x), (res.y - temp.y), (res.z - temp.z), res.visibility] for res in result.pose_landmarks.landmark])
    else:
        landmarks = np.zeros(33*4)

    processed["pose"] = landmarks.flatten()

    if result.face_landmarks:
        temp = list(result.face_landmarks.landmark)[0]
        landmarks = np.array([[(res.x - temp.x), (res.y - temp.y), (res.z - temp.z)] for res in result.face_landmarks.landmark])
    else:
        landmarks = np.zeros(468*3)

    processed["face"] = landmarks.flatten()

    if result.right_hand_landmarks:
        temp = list(result.right_hand_landmarks.landmark)[0]
        landmarks = np.array([[(res.x - temp.x), (res.y - temp.y), (res.z - temp.z)] for res in result.right_hand_landmarks.landmark])
    else:
        landmarks = np.zeros(21*3)

    processed["right_hand"] = landmarks.flatten()

    if result.left_hand_landmarks:
        temp = list(result.left_hand_landmarks.landmark)[0]
        landmarks = np.array([[(res.x - temp.x), (res.y - temp.y), (res.z - temp.z)] for res in result.left_hand_landmarks.landmark])
    else:
        landmarks = np.zeros(21*3)

    processed["left_hand"] = landmarks.flatten()

    return processed
    #return np.concatenate([processed["pose"], processed["face"], processed["left_hand"], processed["right_hand"]])

In [7]:
video = cv2.VideoCapture(0)
# cv2.namedWindow('test')
# cv2.createTrackbar('Brightness', 'test', 255, 2 * 255, BrightnessContrast) 
# cv2.createTrackbar('Contrast', 'test', 127, 2 * 127, BrightnessContrast)

with mpsolution.Holistic(min_detection_confidence = 0.5, min_tracking_confidence=0.5) as model:
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        result1 = detection(frame, model)
        effect = BrightnessContrast(frame, 357, 209)
        result2 = detection(effect, model)

        Superimpose(frame, result1)
        Superimpose(effect, result2)
        
        cv2.imshow("OnlyFans!!", frame)
        cv2.imshow("effect", effect)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

In [23]:
test = Process(result2)
result2

mediapipe.python.solution_base.SolutionOutputs

(1662,)